# Library

In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os

# File Load

In [1]:
ods_fold = 'D:/data/big2/ODS'

In [6]:
# about 2m 28s
file_name = '[ODS][G3]등록정보&제원정보(말소제거)(2022.12.29).csv'
df = pd.read_csv(os.path.join(ods_fold, file_name), encoding='cp949', low_memory=False)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8759431 entries, 0 to 8759430
Data columns (total 49 columns):
 #   Column        Dtype  
---  ------        -----  
 0   차량관리번호        object 
 1   차량등록번호        object 
 2   본거지법정동코드      int64  
 3   소유자구분         object 
 4   차종_등록정보       object 
 5   용도            object 
 6   현재회원번호        object 
 7   차대번호          object 
 8   최초등록일자        int64  
 9   차량연식          int64  
 10  차량제작일자        float64
 11  배출가스검사유효일자    float64
 12  제원관리번호        object 
 13  배출가스인증번호      object 
 14  배출가스등급        object 
 15  저공해조치YN(미사용)  object 
 16  저공해조치종류       float64
 17  저공해조치구조변경YN   object 
 18  차량등록번호숫자정보    int64  
 19  차량말소YN        object 
 20  최초회원번호        object 
 21  취득일자          float64
 22  취득금액          float64
 23  생성일시_등록정보     object 
 24  수정일시_등록정보     object 
 25  차명            object 
 26  차종_제원정보       object 
 27  차종분류          object 
 28  차종유형(소분류)     object 
 29  자동차형식         object 
 30  제작사명          obje

In [4]:
df.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '현재회원번호',
       '차대번호', '최초등록일자', '차량연식', '차량제작일자', '배출가스검사유효일자', '제원관리번호', '배출가스인증번호',
       '배출가스등급', '저공해조치YN(미사용)', '저공해조치종류', '저공해조치구조변경YN', '차량등록번호숫자정보',
       '차량말소YN', '최초회원번호', '취득일자', '취득금액', '생성일시_등록정보', '수정일시_등록정보', '차명',
       '차종_제원정보', '차종분류', '차종유형(소분류)', '자동차형식', '제작사명', '연료', '엔진형식', '공차중량',
       '총중량', '적재중량', '엔진출력', '배기량', '승차인원', '차량길이', '차량너비', '차량높이', '원산지명',
       '삭제YN', '승인일자', '구동형식', '변속기종류', '생성일시_제원정보', '수정일시_제원정보'],
      dtype='object')

In [7]:
df.shape

(8759431, 49)

# 전처리
- 차대번호 중복 제거
    - 최초등록일자, 배출가스검사유효일자 순

In [8]:
df1 = df.sort_values(['최초등록일자', '배출가스검사유효일자'], ascending=[False, False]).drop_duplicates('차대번호').reset_index(drop=True)
df1.shape

(8759393, 49)

In [9]:
df1['배출가스인증번호'].isnull().sum()

730123

In [11]:
count = 0
for one in df1['배출가스인증번호']:
    if ' ' == str(one) or ' ' in str(one):
        count += 1
count

0

In [12]:
df1.groupby('배출가스인증번호', dropna=False)['차대번호'].count()

배출가스인증번호
10MY-PP-14-02         1
10MY-PP-14-03         1
10MY-PP-14-04         1
10MY-PP-14-05         1
10MY-PP-14-W01        1
                  ...  
YMY-KM-17          1562
YMY-KM-18           421
YMY-SM-01         26826
YMY-SM-02         10608
YMY-SM-04          7740
Name: 차대번호, Length: 1538, dtype: int64

In [13]:
df1['연료'].value_counts(dropna=False)

경유             7584050
휘발유             898428
LGP(액화석유가스)     236585
경유 하이브리드         34191
CNG(압축천연가스)       5998
NaN                138
기타연료                 3
Name: 연료, dtype: int64

# 분석
- 3등급 경유차량 배인번호별 차량 대수

## 3등급 경유차량 추출

In [14]:
df1_diesel = df1[df1['연료'] == '경유'].reset_index(drop=True)
df1_diesel.shape

(7584050, 49)

In [16]:
df1_diesel_by_gasnm = df1_diesel.groupby('배출가스인증번호', as_index=False, dropna=False)['차대번호'].count()
df1_diesel_by_gasnm

,배출가스인증번호,차대번호
0,5MY-AD-13-07,119
1,5MY-AD-13-13,11
2,6MY-AD-13-11,323
3,6MY-GK-14-01,11
4,6MY-MK-13-02,498
...,...,...
918,NEG-THD-SR-OM-14-0-1567,1
919,NEG-THD-TM-OM-14-0-1440,2
920,NMY-HD-14-05,43
921,YMY-HD-42,1


In [19]:
df1_diesel_by_gasnm2 = df1_diesel_by_gasnm.sort_values('차대번호', ascending=False).reset_index(drop=True)
df1_diesel_by_gasnm2

,배출가스인증번호,차대번호
0,NaN,591102
1,BMY-HD-14-57,338552
2,GMY-HD-14-36,291636
3,CMY-HD-14-11,252484
4,KMY-HD-14-59,218609
...,...,...
918,NEG-TED-GD-OM-14-0-2039,1
919,NEG-TED-GD-OM-14-0-2214,1
920,NEG-TED-HC-OM-14-0-1605,1
921,NEG-TED-HH-OM-14-0-1413,1


In [21]:
df1_diesel_by_gasnm2 = df1_diesel_by_gasnm2.rename(columns={'차대번호':'대수'})
df1_diesel_by_gasnm2

,배출가스인증번호,대수
0,NaN,591102
1,BMY-HD-14-57,338552
2,GMY-HD-14-36,291636
3,CMY-HD-14-11,252484
4,KMY-HD-14-59,218609
...,...,...
918,NEG-TED-GD-OM-14-0-2039,1
919,NEG-TED-GD-OM-14-0-2214,1
920,NEG-TED-HC-OM-14-0-1605,1
921,NEG-TED-HH-OM-14-0-1413,1


In [31]:
df1_diesel_by_gasnm2.to_excel('analysis2/4등급_데이터_점검/[통계]3등급_경유_배출가스인증번호별_대수.xlsx', index=False)

### 배인번호별 샘플 추출

In [29]:
df2 = df1.copy()
df2['배출가스인증번호'] = df2['배출가스인증번호'].astype('str')

In [34]:
# about 
for one in tqdm(df2['배출가스인증번호']):
    df2[df2['배출가스인증번호'] == one].reset_index(drop=True).to_csv(f'analysis2/4등급_데이터_점검/G3/[샘플]{one}.csv', index=False, encoding='cp949')

# 코드 마지막